In [ ]:
!pip install altair vega_datasets

# Destilacion de Logits

In [ ]:
import pandas as pd
import altair as alt
import lib.utils.viz.KD as KD

In [ ]:
KD.omniplot(folder="./Cifar10/ResNet101/exp1/students")

In [ ]:




#data=KD.load_data(folder="./Cifar10/ResNet101/exp1/students")
KD.load_and_plot(phase='test',field="acc",center=True)


## Destilacion con features

In [ ]:
import altair as alt
from lib.utils.viz import Feat


In [ ]:
Feat.load_and_plot(folder="./Cifar10/ResNet101/exp5/students",phase="train",field="acc",center=True,scale="symlog")

In [ ]:
Feat.load_and_plot(folder="./Cifar10/ResNet101/exp5/students",phase="test",field="acc")#[ARREGLAR LAYERS EN EXP 5]

In [ ]:
plot(data,"test","acc",scale="symlog",center=True)

In [ ]:
Feat.omniplot(folder="./Cifar10/ResNet101/exp5/students",scale="log")

In [ ]:
accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}

d=dict(list(zip(accs['Model'],accs['ce_%s'%"train"])))



# Dataset Merging

In [ ]:
import pandas as pd
def load_KD_data(folder="./Cifar10/ResNet101/exp1/students"):
    data = pd.read_csv(folder+"/summary.csv") 
    data = data[data["student"] != 'EfficientNetB0']
    if "temp" not in data.columns:
        d_arr=[d.split(",")[0] for d in data["distillation"]]
        t_arr=[float(d.split(",T-")[1]) for d in data["distillation"]]
        data["distillation"]=d_arr
        data["temp"]=t_arr
    return data

drop_cols="test_teacher/acc train_teacher/acc lr	epochs	train_batch_size teacher	test_batch_size resume epoch".split()


d1=load_KD_data(folder="./Cifar10/ResNet101/exp1/students").drop(["test_teacher/acc","train_teacher/acc"],axis=1)
d2=load_KD_data(folder="./Cifar10/ResNet101/exp2/students").drop(drop_cols,axis=1)
d3=load_KD_data(folder="./Cifar10/ResNet101/exp3/students").drop(drop_cols,axis=1)
db=d3.merge(d2,on=['student','distillation','temp'],suffixes=('_R01','_R1')).drop(['Unnamed: 0_R01','Unnamed: 0_R1'],axis=1)
db=db.merge(d1,on=['student','distillation','temp']).drop(['epoch','Unnamed: 0'],axis=1)

In [ ]:
detalle=['test_acc' + s for s in ["","_R1",'_R01']]

alt.Chart(db).mark_point().encode(
    alt.X(alt.repeat("column"), type='quantitative',scale=alt.Scale(zero=False,base=10,type='linear')),
    alt.Y(alt.repeat("row"), type='quantitative',scale=alt.Scale(zero=False,base=10,type='linear')),
    color='student',
    shape='distillation'
).properties(
    width=150,
    height=150
).repeat(
    row=detalle,
    column=detalle
)


In [ ]:
db.head()

In [ ]:
detalle

In [ ]:
db[test_acc_R01	] test_acc_R01